# Wykorzystanie pakietu `nltk`

In [2]:
import re
import pandas as pd
import funcs 
import importlib
import unicodecsv as csv
import pickle
import random
from timeit import default_timer as timer
from datetime import datetime, timedelta
import nltk
import nltk.lm
from nltk.util import pad_sequence
from nltk.util import bigrams
from nltk.util import ngrams
from nltk.util import everygrams
from nltk.lm.preprocessing import pad_both_ends
from nltk.lm.preprocessing import flatten
from nltk import word_tokenize, sent_tokenize 
from nltk.lm.preprocessing import padded_everygram_pipeline

importlib.reload(funcs)
importlib.reload(nltk)

<module 'nltk' from 'C:\\Users\\Mag\\Anaconda3\\lib\\site-packages\\nltk\\__init__.py'>

In [3]:
print(nltk.__version__)

3.4.5


In [4]:
sp_joined = pd.read_csv('00_dane/sp_joined.csv',  sep='|', encoding='UTF-8')

Kilka przykładów zastosowania funkcji generującej n-gramy.

In [30]:
test = [word_tokenize(sent_tokenize(sp_joined['speech_rawer'][0])[0])]

In [45]:
list(bigrams(test[0]))

[('Chciałbym', 'kilka'),
 ('kilka', 'zdań'),
 ('zdań', 'powiedzieć'),
 ('powiedzieć', 'na'),
 ('na', 'temat'),
 ('temat', 'skutków'),
 ('skutków', 'tej'),
 ('tej', 'ustawy'),
 ('ustawy', '.')]

In [46]:
training_ngrams, padded_sentences = padded_everygram_pipeline(2, test)
for ngramlize_sent in training_ngrams:
    print(list(ngramlize_sent))
    print()
print('#############')
list(padded_sentences)

[('<s>',), ('Chciałbym',), ('kilka',), ('zdań',), ('powiedzieć',), ('na',), ('temat',), ('skutków',), ('tej',), ('ustawy',), ('.',), ('</s>',), ('<s>', 'Chciałbym'), ('Chciałbym', 'kilka'), ('kilka', 'zdań'), ('zdań', 'powiedzieć'), ('powiedzieć', 'na'), ('na', 'temat'), ('temat', 'skutków'), ('skutków', 'tej'), ('tej', 'ustawy'), ('ustawy', '.'), ('.', '</s>')]

#############


['<s>',
 'Chciałbym',
 'kilka',
 'zdań',
 'powiedzieć',
 'na',
 'temat',
 'skutków',
 'tej',
 'ustawy',
 '.',
 '</s>']

Przetesumy model na jednej wypowiedzi

In [10]:
text = ''.join(sp_joined['speech_rawer'][:10000])


In [11]:
%%time
tok_text = [list(map(str.lower, word_tokenize(sent))) 
                  for sent in sent_tokenize(text)]

# 10 000 Wall time: 1min 31s

Wall time: 1min 31s


In [50]:
tok_text

[['panie', 'marszałku', '!'],
 ['wysoki', 'sejmie', '!'],
 ['chciałbym',
  'kilka',
  'zdań',
  'powiedzieć',
  'na',
  'temat',
  'skutków',
  'tej',
  'ustawy',
  '.'],
 ['moim',
  'zdaniem',
  'podjęcie',
  'tej',
  'inicjatywy',
  'było',
  'jak',
  'najbardziej',
  'zasadne',
  ',',
  'konieczne',
  'i',
  'należy',
  'raz',
  'na',
  'zawsze',
  'zamknąć',
  'to',
  'w',
  'naszej',
  'historii',
  'stanowczym',
  'stanowiskiem',
  'tego',
  'sejmu',
  '.'],
 ['jestem',
  'jednak',
  'za',
  'stwierdzeniem',
  'takim',
  ',',
  'jakie',
  'wnioskuje',
  'komisja',
  ',',
  'że',
  'stan',
  'wojenny',
  'był',
  'bezzasadny',
  ',',
  'że',
  'decyzja',
  'o',
  'wprowadzeniu',
  'stanu',
  'wojennego',
  'została',
  'podjęta',
  'z',
  'naruszeniem',
  'prawa',
  '.'],
 ['wysoka', 'izbo', '!'],
 ['przyjęcie',
  'konstrukcji',
  'innej',
  ',',
  'że',
  'stan',
  'wojenny',
  'był',
  'nielegalny',
  ',',
  'rodzi',
  'nie',
  'tylko',
  'te',
  'konsekwencje',
  ',',
  'o',
  

In [12]:
train_data, padded_sents = padded_everygram_pipeline(2, tok_text)

In [33]:
padded_sents

In [13]:
from nltk.lm import MLE
model = MLE(2)

In [14]:
%%time
model.fit(train_data, padded_sents)

#10 000 Wall time: 3min 51s

Wall time: 3min 51s


In [15]:
print(model.counts)

<NgramCounter with 3 ngram orders and 19163778 ngrams>


In [43]:
model.counts['<\s>']

0

In [42]:
model.counts[['<\s>']]['<\s>']

0

In [18]:
model.counts[['moim','zdaniem']]['podjęcie']

1

In [44]:
print(model.generate(30, text_seed=['<s>']))

['projekt', 'platformy', 'obywatelskiej', ',', 'prawa', 'socjalne', ',', 'wysłali', 'do', 'hagi', 'list', 'wzywający', 'holenderski', 'rząd', 'do', 'kompleksowego', 'ujęcia', '.', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>']


In [37]:
model.score('gwarantować', 'zabezpieczenia mają'.split())

1.0